In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [3]:
! curl -O http://www.gutenberg.org/files/1268/1268-0.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1144k  100 1144k    0     0   725k      0  0:00:01  0:00:01 --:--:--  725k


In [4]:
with open('/content/1268-0.txt', 'r') as fp:
    text=fp.read()

start_indx = text.find('THE MYSTERIOUS ISLAND')
end_indx = text.find('End of the Project Gutenberg')
print(start_indx, end_indx)

text = text[start_indx:end_indx]
char_set = set(text)
print('Total Length:', len(text))
print('Unique Characters:', len(char_set))

567 1112917
Total Length: 1112350
Unique Characters: 80


In [5]:
chars_sorted = sorted(char_set)
char2int = {ch:i for i,ch in enumerate(chars_sorted)}
char_array = np.array(chars_sorted)

text_encoded = np.array(
    [char2int[ch] for ch in text], dtype=np.int32)

print('Text encoded shape: ', text_encoded.shape)

print(text[:15], '     ==Encoding==> ', text_encoded[:15])
print(text_encoded[15:21], '  == Reverse ==> ',''.join(char_array[text_encoded[15:21]]))



Text encoded shape:  (1112350,)
THE MYSTERIOUS       ==Encoding==>  [44 32 29  1 37 48 43 44 29 42 33 39 45 43  1]
[33 43 36 25 38 28]   == Reverse ==>  ISLAND


In [6]:
ds_text_encoded = tf.data.Dataset.from_tensor_slices(text_encoded)

for ex in ds_text_encoded.take(5):
    print('{} -> {}'.format(ex.numpy(), char_array[ex.numpy()]))

44 -> T
32 -> H
29 -> E
1 ->  
37 -> M


In [7]:
seq_length = 40
chunk_size = seq_length + 1

ds_chunks = ds_text_encoded.batch(chunk_size, drop_remainder=True)

for seq in ds_chunks.take(1):
    input_seq = seq[:seq_length].numpy()
    target = seq[seq_length].numpy()
    print(input_seq, '->', target)
    print(repr(''.join(char_array[input_seq])),
          '->', repr(''.join(char_array[target])))

[44 32 29  1 37 48 43 44 29 42 33 39 45 43  1 33 43 36 25 38 28  1  6  6
  6  0  0  0  0  0 40 67 64 53 70 52 54 53  1 51] -> 74
'THE MYSTERIOUS ISLAND ***\n\n\n\n\nProduced b' -> 'y'


In [8]:
def split_input_target(chunk):
    input_seq = chunk[:-1]
    target_seq = chunk[1:]
    return input_seq, target_seq

ds_sequences = ds_chunks.map(split_input_target)

for example in ds_sequences.take(2):
    print('Input (x):', repr(''.join(char_array[example[0].numpy()])))
    print('Target (y):', repr(''.join(char_array[example[1].numpy()])))
    print()

Input (x): 'THE MYSTERIOUS ISLAND ***\n\n\n\n\nProduced b'
Target (y): 'HE MYSTERIOUS ISLAND ***\n\n\n\n\nProduced by'

Input (x): ' Anthony Matonak, and Trevor Carlson\n\n\n\n'
Target (y): 'Anthony Matonak, and Trevor Carlson\n\n\n\n\n'



In [9]:
batch_size = 64
buffer_size = 10000

tf.random.set_seed(1)
ds = ds_sequences.shuffle(buffer_size).batch(batch_size)

ds

<BatchDataset shapes: ((None, 40), (None, 40)), types: (tf.int32, tf.int32)>

In [26]:


def build_model(vocab_size, embedding_dim, rnn_units):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim),
        tf.keras.layers.LSTM(
            rnn_units, return_sequences=True),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model


charset_size = len(char_array)
embedding_dim = 256
rnn_units = 512

tf.random.set_seed(1)

model = build_model(
    vocab_size = charset_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 256)         20480     
_________________________________________________________________
lstm_1 (LSTM)                (None, None, 512)         1574912   
_________________________________________________________________
dense_1 (Dense)              (None, None, 80)          41040     
Total params: 1,636,432
Trainable params: 1,636,432
Non-trainable params: 0
_________________________________________________________________


In [27]:
model.compile(
    optimizer='adam', 
    loss=tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=True
    ))

model.fit(ds, epochs=20)

Epoch 1/20
424/424 [==============================] - 6s 14ms/step - loss: 2.3237
Epoch 2/20
424/424 [==============================] - 6s 14ms/step - loss: 1.7470
Epoch 3/20
424/424 [==============================] - 6s 14ms/step - loss: 1.5454
Epoch 4/20
424/424 [==============================] - 6s 14ms/step - loss: 1.4291
Epoch 5/20
424/424 [==============================] - 6s 14ms/step - loss: 1.3550
Epoch 6/20
424/424 [==============================] - 6s 14ms/step - loss: 1.3038
Epoch 7/20
424/424 [==============================] - 6s 14ms/step - loss: 1.2651
Epoch 8/20
424/424 [==============================] - 6s 14ms/step - loss: 1.2345
Epoch 9/20
424/424 [==============================] - 6s 14ms/step - loss: 1.2087
Epoch 10/20
424/424 [==============================] - 6s 14ms/step - loss: 1.1872
Epoch 11/20
424/424 [==============================] - 6s 14ms/step - loss: 1.1671
Epoch 12/20
424/424 [==============================] - 6s 14ms/step - loss: 1.1501
Epoch 13/20
4

In [16]:
tf.random.set_seed(1)
logits = [[1.0, 1.0, 1.0]]
print('Probabilities:', tf.math.softmax(logits).numpy()[0])

samples = tf.random.categorical(
    logits=logits, num_samples=10)
tf.print(samples.numpy())

Probabilities: [0.33333334 0.33333334 0.33333334]
array([[1, 2, 0, 1, 0, 1, 1, 2, 1, 1]])


In [21]:
tf.random.set_seed(1)

logits = [[1.0, 1.0, 3.0]]
print('Probabilities:', tf.math.softmax(logits).numpy()[0])

samples = tf.random.categorical(
    logits=logits, num_samples=10)
tf.print(samples.numpy())



Probabilities: [0.10650698 0.10650698 0.78698605]
array([[2, 2, 0, 2, 2, 2, 2, 2, 1, 2]])
Probabilities: [0.10650698 0.10650698 0.78698605]
array([[2, 2, 0, 2, 2, 2, 2, 2, 1, 2]])


In [28]:
def sample(model, starting_str,
           len_generated_text=500,
           max_input_length=40,
           scale_factor=1.0):
    encoded_input = [char2int[s] for s in starting_str]
    encoded_input = tf.reshape(encoded_input, (1, -1))

    generated_str = starting_str

    model.reset_states()
    for i in range(len_generated_text):
        logits = model(encoded_input)
        logits = tf.squeeze(logits, 0)

        scaled_logits = logits * scale_factor
        new_char_indx = tf.random.categorical(
            scaled_logits, num_samples=1)
        
        new_char_indx = tf.squeeze(new_char_indx)[-1].numpy()

        generated_str += str(char_array[new_char_indx])

        new_char_indx = tf.expand_dims([new_char_indx], 0)
        encoded_input = tf.concat(
            [encoded_input, new_char_indx],
            axis=1)
        encoded_input = encoded_input[:, -max_input_length:]
    
    return generated_str

tf.random.set_seed(1)
print(sample(model, starting_str='The island'))

The island was extremely still from the latter necessary.

“Captain Nemo,” cried Pencroft, “but are the crater Jus of those valleys repailor. “We have your you,” answered Cyrus Harding.

“Gentleming came,” said the sail,
ravage
barking--” said the edge shaded to perish to our arrows. It had
found no easily of fieldetile that the country, began to believe that he was, Pencroft and Jup then set to him. What actoining fourpoither obstacle so fell from excellent
eruption, which swept this last sea-border out 


In [31]:
tf.random.set_seed(1)
print(sample(model, starting_str='The island',
             scale_factor=2.0))

The island was extremely surprise.

The colonists had a consisted and found at the next day, to the beach. It was decided that the hunters were soon of the coast. The colonists had near the coast. The reporter and Herbert had been succeeded in the first stream of the gloom. How he should not be able to continue to the engineer, “and we shall not do as to see if it did not appear to cross the midst of the albatross, and proceeded towards the surface, and the wire was there to be able to difficulty.

The co


In [32]:
tf.random.set_seed(1)
print(sample(model, starting_str='The island',
             scale_factor=0.5))

The island had egblopped this kinnws,

“He” young yals! ciffom. He never, I
rejoke,--1 of
Joctom of baguar-mace,
supportohes
abonavh Pal;
unflicintiousy,
sin hepot; Ayrton now; Prospear 1Y. I did or,”!

“We can, at Fawl,
regard
you; you,” exhouren
Spilett rock to ester. Mood and fest,! a palf of!” cridihed Cyaf excetilet. Yes: lithustocscot,
of see--Hostes Pencroft no last as far ancro-hishes relun. The “alcaliar,
yet we
give,” exclaided Pencroft; very falt;
“Su-fall,” replied Harding.

Atrisant JulyAgue 
